In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [46]:
import tensorflow as tf
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Dense, Dropout, add
from tensorflow.keras.optimizers import Adam, SGD

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

tf.__version__

'2.1.0'

In [3]:
# datasets
housing = pd.read_csv('./kc_house_data.csv')
slmnist = pd.read_csv('./slmnist.csv', header=None)

In [4]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Defining neural networks with `keras`

## The sequential model in `keras`

In [6]:
# define a Keras sequential model
model = Sequential()

# define the layers
model.add(Dense(16, activation='relu', input_shape=(784,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax')) # output layer

# print the model architecture
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                12560     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
Total params: 12,732
Trainable params: 12,732
Non-trainable params: 0
_________________________________________________________________
None


## Compiling a sequential model

In [8]:
# define a sequential model
model = Sequential()

# define the layers
model.add(Dense(16, activation='sigmoid', input_shape=(784,)))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax')) # output

# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy')

# print model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                12560     
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 68        
Total params: 12,628
Trainable params: 12,628
Non-trainable params: 0
_________________________________________________________________
None


## Defining a multiple input model

In [10]:
# define input layers
m1_inputs = Input(shape=(784,))
m2_inputs = Input(shape=(784,))

In [15]:
# for model 1, pass the input layer to layer 1 and layer 1 to layer 2
m1_layer1 = Dense(12, activation='sigmoid')(m1_inputs)
m1_layer2 = Dense(4, activation='softmax')(m1_layer1)

# for model 2, do the same
m2_layer1 = Dense(12, activation='relu')(m2_inputs)
m2_layer2 = Dense(4, activation='softmax')(m2_layer1)

# merge model outputs and define a functional model
merged = add([m1_layer2, m2_layer2])
model = Model(inputs=[m1_inputs, m2_inputs],
              outputs=merged)

# print a model summary
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 12)           9420        input_1[0][0]                    
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 12)           9420        input_2[0][0]                    
______________________________________________________________________________________________

# Training and validation with `keras`

## Training with `keras`

In [18]:
print(slmnist.shape)
slmnist.head(3)

(2000, 785)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,1,142,143,146,148,149,149,149,150,151,151,151,151,150,152,151,150,149,147,146,145,143,142,140,139,137,135,133,131,144,145,147,150,150,151,153,153,153,154,153,153,152,151,161,152,145,153,150,149,146,144,143,143,141,137,136,133,145,148,150,152,153,154,155,155,155,156,156,155,157,152,173,168,114,147,149,152,150,147,145,144,142,140,138,137,148,150,152,154,155,155,156,157,158,158,159,156,168,166,131,177,92,126,128,151,152,150,149,145,143,142,141,139,150,152,154,156,157,157,159,159,161,161,162,156,180,178,107,181,107,141,121,135,157,152,150,148,146,145,143,141,151,155,156,157,159,160,161,162,164,162,164,156,184,190,110,184,123,137,138,124,162,153,152,150,149,148,146,144,154,157,159,160,162,163,164,164,165,163,183,140,170,193,110,163,120,144,157,119,163,156,155,153,152,151,149,146,155,159,162,164,163,164,165,167,167,167,199,130,161,183,104,152,117,148,155,115,163,158,158,156,154,153,151,148,157,161,163,165,165,167,168,169,168,172,207,132,168,194,108,159,126,14

In [33]:
# sign language mnist dataset (4 targets)
sign_X = slmnist.iloc[:, 1:]
sign_y = pd.get_dummies(slmnist.iloc[:, 0]).to_numpy(np.float32)

sign_X_train, sign_X_test, sign_y_train, sign_y_test = train_test_split(sign_X, sign_y,
                                                                        test_size=0.5,
                                                                        random_state=42,
                                                                        stratify=sign_y)

# scale data
mm = MinMaxScaler()
sign_X_train = mm.fit_transform(sign_X_train)
sign_X_test = mm.transform(sign_X_test)

In [40]:
# define a sequential model
model = Sequential()

# define layers
model.add(Dense(16, activation='relu', input_shape=(784,)))
model.add(Dense(4, activation='softmax')) #output

# compile
model.compile(optimizer='SGD',
              loss='categorical_crossentropy')

# fit
model.fit(sign_X_train, sign_y_train,
          epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 5s 5ms/sample - loss: 1.2999
Epoch 2/5
1000/1000 [==============================] - 0s 141us/sample - loss: 1.1077
Epoch 3/5
1000/1000 [==============================] - 0s 132us/sample - loss: 0.9580
Epoch 4/5
1000/1000 [==============================] - 0s 136us/sample - loss: 0.8413
Epoch 5/5
1000/1000 [==============================] - 0s 137us/sample - loss: 0.7433


You probably noticed that your only measure of performance improvement was the value of the loss function in the training sample, which is not particularly informative. You will improve on this in the next exercise.

## Metrics and validation with `keras`

In [43]:
# define a model architecture
model = Sequential()

model.add(Dense(32, activation='sigmoid', input_shape=(784,)))
model.add(Dense(4, activation='softmax'))

# compile and fit
model.compile(optimizer='RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(sign_X_train, sign_y_train,
          epochs=10,
          validation_split=0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/10
900/900 [==============================] - 1s 733us/sample - loss: 1.2677 - accuracy: 0.4500 - val_loss: 1.2283 - val_accuracy: 0.4400
Epoch 2/10
900/900 [==============================] - 0s 201us/sample - loss: 0.9892 - accuracy: 0.6744 - val_loss: 1.0456 - val_accuracy: 0.5700
Epoch 3/10
900/900 [==============================] - 0s 195us/sample - loss: 0.8210 - accuracy: 0.7489 - val_loss: 0.8871 - val_accuracy: 0.5600
Epoch 4/10
900/900 [==============================] - 0s 194us/sample - loss: 0.7035 - accuracy: 0.8133 - val_loss: 0.9258 - val_accuracy: 0.6500
Epoch 5/10
900/900 [==============================] - 0s 198us/sample - loss: 0.6077 - accuracy: 0.8600 - val_loss: 0.5713 - val_accuracy: 0.8100
Epoch 6/10
900/900 [==============================] - 0s 203us/sample - loss: 0.5192 - accuracy: 0.8922 - val_loss: 0.7564 - val_accuracy: 0.6100
Epoch 7/10
900/900 [==============================] - 0s 193us/sample - loss: 

## Overfitting detection

In [45]:
# define model architecture
model = Sequential()

model.add(Dense(1024, activation='relu', input_shape=(784,)))
model.add(Dense(4, activation='softmax'))

# compile and fit
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(sign_X_train, sign_y_train,
          epochs=50,
          validation_split=0.5)

Train on 500 samples, validate on 500 samples
Epoch 1/50
500/500 [==============================] - 1s 1ms/sample - loss: 1.6931 - accuracy: 0.4140 - val_loss: 0.9916 - val_accuracy: 0.4820
Epoch 2/50
500/500 [==============================] - 0s 391us/sample - loss: 0.7695 - accuracy: 0.7040 - val_loss: 0.5624 - val_accuracy: 0.7680
Epoch 3/50
500/500 [==============================] - 0s 426us/sample - loss: 0.4786 - accuracy: 0.8600 - val_loss: 0.4617 - val_accuracy: 0.8600
Epoch 4/50
500/500 [==============================] - 0s 384us/sample - loss: 0.3479 - accuracy: 0.9540 - val_loss: 0.3075 - val_accuracy: 0.9460
Epoch 5/50
500/500 [==============================] - 0s 390us/sample - loss: 0.2618 - accuracy: 0.9760 - val_loss: 0.2771 - val_accuracy: 0.9080
Epoch 6/50
500/500 [==============================] - 0s 378us/sample - loss: 0.2210 - accuracy: 0.9660 - val_loss: 0.2226 - val_accuracy: 0.9540
Epoch 7/50
500/500 [==============================] - 0s 366us/sample - loss: 0.

## Evaluating models

In [47]:
# define 2 models - small and large

small_model = tf.keras.Sequential()

small_model.add(Dense(8, activation='relu', input_shape=(784,)))
small_model.add(Dense(4, activation='softmax'))

small_model.compile(optimizer=SGD(lr=0.01), 
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])


large_model = tf.keras.Sequential()

large_model.add(Dense(64, activation='sigmoid', input_shape=(784,)))
large_model.add(Dense(4, activation='softmax'))

large_model.compile(optimizer=Adam(learning_rate=0.001, 
                                   beta_1=0.9, 
                                   beta_2=0.999),
                    loss='categorical_crossentropy', 
                    metrics=['accuracy'])

In [49]:
small_model.fit(sign_X_train, sign_y_train,
                epochs=20,
                verbose=False)
large_model.fit(sign_X_train, sign_y_train,
                epochs=20, 
                verbose=False)

In [54]:
# evaluate models using train and test data
small_train = small_model.evaluate(sign_X_train, sign_y_train)
small_test = small_model.evaluate(sign_X_test, sign_y_test)
large_train = large_model.evaluate(sign_X_train, sign_y_train)
large_test = large_model.evaluate(sign_X_test, sign_y_test)

# print losses
print(f'\nsmall_model loss on train: {small_train[0]} on test: {small_test[0]}')
print(f'\nlarge_model loss on train: {large_train[0]} on test: {large_test[0]}')

1000/1000 [==============================] - 0s 119us/sample - loss: 0.0507 - accuracy: 0.9990

small_model loss on train: 0.16714009022712709 on test: 0.17680063104629518

large_model loss on train: 0.04768511414527893 on test: 0.050714456140995026


# Training models with the Estimators API

## Preparing to train with Estimators

In [55]:
# define feature columns for bedrooms and bathrooms
bedrooms = tf.feature_column.numeric_column('bedrooms')
bathrooms = tf.feature_column.numeric_column('bathrooms')

# define list of feature columns
feature_list = [bedrooms, bathrooms]

def input_fn():
    # define labels
    labels = housing['price'].to_numpy()
    
    # define features
    features = {'bedrooms': housing['bedrooms'].to_numpy(),
                'bathrooms': housing['bathrooms'].to_numpy()}
    
    return features, labels

## Defining Estimators

In [56]:
# define the model and set the number of steps
model = tf.estimator.DNNRegressor(feature_columns=feature_list,
                                  hidden_units=[2, 2])
model.train(input_fn, steps=1)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\loujo\\AppData\\Local\\Temp\\tmp2y65e6ex', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
I

In [57]:
# modify model
model = tf.estimator.LinearRegressor(feature_columns=feature_list)
model.train(input_fn, steps=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\loujo\\AppData\\Local\\Temp\\tmpesxdapbt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, y